In [1]:
using Pkg
Pkg.activate(raw"libs/")
# Pkg.instantiate()
# Pkg.add("ImageView")
using Lux, NNlib, Zygote, LuxCUDA, CUDA, JLD2, DICOM
using Images, ImageView
using MLUtils
using Optimisers, ProgressBars
using ImageMorphology, ChainRulesCore, Statistics
using Plots

CUDA.allowscalar(false)

valid_data_dir = raw"C:\Users\wenbl13\Desktop\BAC\collected_dataset_for_ML\valid"
saved_model_dir = raw"C:\Users\wenbl13\Desktop\BAC\BAC Project"
output_pred_dir = raw"C:\Users\wenbl13\Desktop\BAC\BAC Project\Output"
output_images_dir = raw"C:\Users\wenbl13\Desktop\BAC\BAC Project\images_Output"
patch_size = 256
patch_size_half = round(Int, patch_size/2);

  Activating project at `~/Desktop/Project BAC/BAC project/libs`


Gtk-Message: 15:00:59.076: Failed to load module "canberra-gtk-module"
Gtk-Message: 15:00:59.076: Failed to load module "canberra-gtk-module"


In [2]:
using CSV, DataFrames

# Helper functions

In [2]:
_conv = (in, out) -> Conv((3, 3), in=>out, pad=1)

conv1 = (in, out) -> Chain(_conv(in, out), BatchNorm(out, leakyrelu))
# conv2 = (in, out) -> Chain(_conv(in, out), x -> softmax(x; dims = 3))
conv2 = (in, out) -> Chain(Conv((1, 1), in=>out), sigmoid)

_tran = (in, out) -> ConvTranspose((2, 2), in => out, stride = 2)
tran = (in, out) -> Chain(_tran(in, out), BatchNorm(out, leakyrelu))

my_cat = (x, y) -> cat(x, y; dims=Val(3))

function unet2D(in_chs, lbl_chs)    
    # Contracting layers
    l1 = Chain(conv1(in_chs, 64), conv1(64, 64))
    l2 = Chain(l1, MaxPool((2,2), stride=2), conv1(64, 128), conv1(128, 128))
    l3 = Chain(l2, MaxPool((2,2), stride=2), conv1(128, 256), conv1(256, 256))
    l4 = Chain(l3, MaxPool((2,2), stride=2), conv1(256, 512), conv1(512, 512))
    l5 = Chain(l4, MaxPool((2,2), stride=2), conv1(512, 1024), conv1(1024, 1024), tran(1024, 512))
    
    # Expanding layers
    l6 = Chain(Parallel(my_cat,l5,l4), conv1(512+512, 512), conv1(512, 512), tran(512, 256))
    l7 = Chain(Parallel(my_cat,l6,l3), conv1(256+256, 256), conv1(256, 256), tran(256, 128))
    l8 = Chain(Parallel(my_cat,l7,l2), conv1(128+128, 128), conv1(128, 128), tran(128, 64))
    l9 = Chain(Parallel(my_cat,l8,l1), conv1(64+64, 64), conv1(64, 64), conv2(64, lbl_chs))
end

function zoom_pxiel_values(img)
    a, b = minimum(img), maximum(img)
    img_ = (img .- a) ./ (b - a)
    return img_
end

function normalize_img(img)
    m = maximum(img)
    img = m .- img
    a = mean(img)
    s = std(img)
    img = (img .- a) ./ s 
    # println("mean = $(mean(img)), std = $(std(img))")
    return img
end

function get_num_of_forgound_patches(mask, s)
    x = ceil(Int, s[1]/patch_size) + floor(Int, (s[1]-patch_size_half)/patch_size)
    y = ceil(Int, s[2]/patch_size) + floor(Int, (s[2]-patch_size_half)/patch_size)
    ct = 0
    for i = 1 : x-1
        x_start = 1+(i-1)*patch_size_half
        x_end = x_start+patch_size-1
        for j = 1 : y-1
            y_start = 1+(j-1)*patch_size_half
            y_end = y_start+patch_size-1
            # check patch
            if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
                ct += 1
            end
        end
        # right col
        y_start, y_end = s[2]-patch_size+1, s[2]
        # check patch
        if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
            ct += 1
        end
    end
    # last row
    x_start, x_end = s[1]-patch_size+1, s[1]
    for j = 1 : y-1
        y_start = 1+(j-1)*patch_size_half
        y_end = y_start+patch_size-1
        # check patch
        if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
            ct += 1
        end
    end
    # right col
    y_start, y_end = s[2]-patch_size+1, s[2]
    # check patch
    if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
        ct += 1
    end
    return ct
end

function patch_image(img, mask)
    # img_max = maximum(img)
    # img = img_max .- img
    img = normalize_img(img)
    s = size(img)
    num_patches = get_num_of_forgound_patches(mask, s)
    # @info "num_patches = $num_patches"
    x = ceil(Int, s[1]/patch_size) + floor(Int, (s[1]-patch_size_half)/patch_size)
    y = ceil(Int, s[2]/patch_size) + floor(Int, (s[2]-patch_size_half)/patch_size)

    img_patches = Array{Float32, 4}(undef, patch_size, patch_size, 1, num_patches)
    locations = Array{Tuple{Int64, Int64, Int64, Int64}, 1}(undef, num_patches)
    ct = 0
    for i = 1 : x-1
        x_start = 1+(i-1)*patch_size_half
        x_end = x_start+patch_size-1
        for j = 1 : y-1
            y_start = 1+(j-1)*patch_size_half
            y_end = y_start+patch_size-1
            # check patch
            if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
                # save patch
                ct += 1
                img_patches[:, :, 1, ct] = (img[x_start:x_end, y_start:y_end])
                locations[ct] = (x_start, x_end, y_start, y_end)
            end
        end
        # right col
        y_start, y_end = s[2]-patch_size+1, s[2]
        # check patch
        if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
            # save patch
            ct += 1
            img_patches[:, :, 1, ct] = (img[x_start:x_end, y_start:y_end])
            locations[ct] = (x_start, x_end, y_start, y_end)
        end
    end
    # last row
    x_start, x_end = s[1]-patch_size+1, s[1]
    for j = 1 : y-1
        y_start = 1+(j-1)*patch_size_half
        y_end = y_start+patch_size-1
        # check patch
        if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
            # save patch
            ct += 1
            img_patches[:, :, 1, ct] = (img[x_start:x_end, y_start:y_end])
            locations[ct] = (x_start, x_end, y_start, y_end)
        end
    end
    # right col
    y_start, y_end = s[2]-patch_size+1, s[2]
    # check patch
    if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
        # save patch
        ct += 1
        img_patches[:, :, 1, ct] = (img[x_start:x_end, y_start:y_end])
        locations[ct] = (x_start, x_end, y_start, y_end)
    end
    # return
    return img_patches, locations, num_patches
end

patch_image (generic function with 1 method)

In [3]:
function pred_img(img_in, model_info)
    # get binary mask
    # @info "Getting binary mask..."
    img = deepcopy(img_in)
    img_out = zeros(Float32, size(img))
    img_mask = 1 .- round.(zoom_pxiel_values(img_in))
    
    # patch img_in
    # @info "Patching image..."
    img_patches, locations, num_patches = patch_image(img, img_mask)
    pred_patches = Array{Float32, 3}(undef, patch_size, patch_size, num_patches)

    # load model
    # @info "Loading model..."
    model, ps, st = model_info

    # apply model on all patches
    # @info "Running predictions..."
    for i = 1 : num_patches
        ignore_derivatives() do
            curr_patch = (img_patches[:,:,:,i:i]) |> gpu # 256*256*1*1
            curr_pred = model(curr_patch, ps, st)[1] |> cpu # 256*256*2*1
            pred_patches[:,:,i] = curr_pred[:,:,1,1]
        end
    end

    # combines patch to pred_out
    # @info "Gathering output"
    Threads.@threads for i = 1 : num_patches
        x_start, x_end, y_start, y_end = locations[i]
        for x = x_start : x_end
            for y = y_start : y_end
                img_out[x,y] = pred_patches[x-x_start+1,y-y_start+1,i]>=0.5 ? 1f0 : img_out[x,y]
                # img_out[x,y] += pred_patches[x-x_start+1,y-y_start+1,i]
            end
        end
    end
    
    return img_out, pred_patches
end

pred_img (generic function with 1 method)

In [4]:
function pick_all_valid_images(valid_dir)
    picked_images = []
    image_dir = joinpath(valid_dir,"image")
    label_dir = joinpath(valid_dir,"label")
    SIDs = readdir(image_dir)
    for SID in SIDs
        # set dir
        curr_dir = joinpath(label_dir, SID)
        if !isdir(curr_dir)
            continue
        end
        # read files
        for f in readdir(curr_dir)
            f_name, f_ext = splitext(f)
            if f_ext == ".png"
                curr_png = Float32.(Images.load(joinpath(curr_dir, f)))
                img_path = joinpath(image_dir, SID, f_name*".dcm")
                dcm_img = Float32.(dcm_parse(img_path)[(0x7fe0, 0x0010)])
                push!(picked_images, [SID, f_name, dcm_img, curr_png])
            end
        end
    end
    return picked_images
end

pick_all_valid_images (generic function with 1 method)

# 1. Run all images

In [6]:
# Read all images(runtime ~= 40s)
picked_images = pick_all_valid_images(valid_data_dir);

In [47]:
# Run through all images(runtime ~= 37min for 200 images)
GC.gc(true)
epoch_idx = 35
num_images = size(picked_images)[1]
pred_results = []
model = unet2D(1, 1)
# load model
model_name = "saved_train_info_$epoch_idx.jld2"
model_path = joinpath(saved_model_dir, model_name)
@load model_path ps_save st_save
ps = ps_save |> gpu
st = st_save |> gpu
model_info = (model, ps, st);

for i = 1 : num_images
    @info i
    SID, f_name, x, label = picked_images[i]
    # get prediction
    pred_image, _ = pred_img(x, model_info)
    # save
    save_dir = joinpath(output_pred_dir, SID)
    isdir(save_dir) || mkdir(save_dir)
    save_path = joinpath(save_dir, f_name*".jld")
    @save save_path pred_image label
    GC.gc(true)
end

┌ Info: 1
└ @ Main c:\Users\wenbl13\Desktop\BAC\BAC Project\4.4_run_all_valid_images.ipynb:16


┌ Info: 2
└ @ Main c:\Users\wenbl13\Desktop\BAC\BAC Project\4.4_run_all_valid_images.ipynb:16


┌ Info: 3
└ @ Main c:\Users\wenbl13\Desktop\BAC\BAC Project\4.4_run_all_valid_images.ipynb:16


┌ Info: 4
└ @ Main c:\Users\wenbl13\Desktop\BAC\BAC Project\4.4_run_all_valid_images.ipynb:16


┌ Info: 5
└ @ Main c:\Users\wenbl13\Desktop\BAC\BAC Project\4.4_run_all_valid_images.ipynb:16


...

┌ Info: 196
└ @ Main c:\Users\wenbl13\Desktop\BAC\BAC Project\4.4_run_all_valid_images.ipynb:16


┌ Info: 197
└ @ Main c:\Users\wenbl13\Desktop\BAC\BAC Project\4.4_run_all_valid_images.ipynb:16


┌ Info: 198
└ @ Main c:\Users\wenbl13\Desktop\BAC\BAC Project\4.4_run_all_valid_images.ipynb:16


┌ Info: 199
└ @ Main c:\Users\wenbl13\Desktop\BAC\BAC Project\4.4_run_all_valid_images.ipynb:16


┌ Info: 200
└ @ Main c:\Users\wenbl13\Desktop\BAC\BAC Project\4.4_run_all_valid_images.ipynb:16


# 2. Save predictions as png

In [48]:
plot_SIDs = readdir(output_pred_dir)
for SID in plot_SIDs
    curr_dir = joinpath(output_pred_dir, SID)
    if isdir(curr_dir)
        for f in readdir(curr_dir)
            # read
            curr_path = joinpath(curr_dir, f)
            @load curr_path pred_image label
            # convert to png
            image_output = Gray.(zoom_pxiel_values(pred_image))
            label_output = Gray.(label)
            # set up path
            f_name, _ = splitext(f)
            image_path = joinpath(output_images_dir, SID, "pred_"*f_name*".png")
            label_path = joinpath(output_images_dir, SID, "roi_"*f_name*".png")
            # save
            save(image_path, image_output)
            save(label_path, label_output)
        end
    end
end
GC.gc(true)

# 3. Plot

In [5]:
plot_SIDs = readdir(output_pred_dir)
areas_ = []
for SID in plot_SIDs
    curr_dir = joinpath(output_pred_dir, SID)
    if isdir(curr_dir)
        for f in readdir(curr_dir)
            curr_path = joinpath(curr_dir, f)
            @load curr_path pred_image label
            push!(areas_, [SID, f, sum(label), sum(pred_image)])
        end
    end
end
s = size(areas_)[1]
areas__ = zeros(Int64, (s, 2))
for i = 1 : s
    areas__[i, :] = Int64.(areas_[i][3:4])
end 
GC.gc(true)

In [6]:
scatter(areas__[:, 1], areas__[:, 2], xlabel="Readers' roi", ylabel="Prediction", title="BAC area")
plot!(areas__[:, 1], areas__[:, 1], label="y = x", linestyle=:dash)
savefig(joinpath(output_pred_dir, "BAC_area_plot2.png"))

"C:\\Users\\wenbl13\\Desktop\\BAC\\BAC Project\\Output\\BAC_area_plot2.png"

In [30]:
idx = []
# Check for points
for i = 1 : s
    label_area, pred_area = areas__[i, :]
    ratio = label_area / pred_area
    if ratio > 4
        println(i, ": ", ratio)
    else
        push!(idx, i)
    end
end

13: 4.646377979960843
14: 9.021660275676236
16: 6.037864016604818
22: 4.426022706794017
23: 4.451520523660166
24: 4.294606143674917
70: 4.576734811402246


13: 4.646377979960843  
14: 9.021660275676236  
16: 6.037864016604818  
22: 4.426022706794017  
23: 4.451520523660166  
24: 4.294606143674917  
70: 4.576734811402246  

In [34]:
areas_output = areas__[idx]
df = DataFrame(X = areas__[:, 1], Y = areas__[:, 2])

# Save the DataFrame to a CSV file
CSV.write("BAC_plot_coordinates.csv", df)

193-element Vector{Int64}:
 46550
 50157
 78838
 76932
 34529
 49156
 29689
 82400
 22467
 48105
     ⋮
 50851
 71300
 66677
 92647
 93028
 38315
 57761
 93617
 72387

In [9]:
areas_[13]

4-element Vector{Any}:
       "SID-103073"
       "1.2.840.113681.2230565208.1099.3632750534.308.jld"
 242067.0f0
  52098.0f0

# 4. Save images as png

In [10]:
plot_SIDs = readdir(output_pred_dir)
for SID in plot_SIDs
    curr_dir = joinpath(output_pred_dir, SID)
    if isdir(curr_dir)
        for f in readdir(curr_dir)
            # setup path
            f_name, _ = splitext(f)
            image_path = joinpath(valid_data_dir, "image", SID, f_name*".dcm")
            save_path = joinpath(output_images_dir, SID, "image_"*f_name*".png")
            # read
            dcm_img = Float32.(dcm_parse(image_path)[(0x7fe0, 0x0010)])
            dcm_img = my_process_img(dcm_img)
            dcm_img = Gray.(dcm_img)
            save
            save(save_path, dcm_img)
        end 
    end
end
GC.gc(true)

UndefVarError: UndefVarError: `my_process_img` not defined

# Fix wrong mask

In [ ]:
SIDs = readdir(output_pred_dir)

In [11]:
function my_process_img(img)
    # get mask
    mask = zoom_pxiel_values(img)
    mask = 1f0 .- round.(mask)
    img_ = img[mask .== 1f0]
    # zoom pixel values to 0 and 1
    a, b = maximum(img_), minimum(img_)
    img_ = (img_ .- b) ./ (a - b)
    img_ = 1f0 .- img_
    # histogram equalization
    img_ = histogram_equalization_(img_)
    img[mask .== 1f0] .= img_
    # save
    img = img .* mask
    return img
end

my_process_img (generic function with 1 method)

In [12]:
function histogram_equalization_(img)
    # Get array length
    len = length(img)
    
    # Initialize histogram and cumulative histogram
    nbins = 256
    hist = zeros(Int, nbins)
    chist = zeros(Int, nbins)
    
    # Compute the histogram
    for val in img
        bin = Int(floor(val * (nbins - 1)) + 1)
        hist[bin] += 1
    end
    
    # Compute the cumulative histogram
    chist[1] = hist[1]
    for i in 2:nbins
        chist[i] = chist[i - 1] + hist[i]
    end
    
    # Perform histogram equalization
    min_chist = minimum(filter(x -> x > 0, chist))
    total_pixels = len
    new_img = zeros(Float32, len)
    
    for i in 1:len
        bin = Int(floor(img[i] * (nbins - 1)) + 1)
        new_intensity = (chist[bin] - min_chist) / (total_pixels - min_chist)
        new_img[i] = Float32(new_intensity)
    end
    
    return new_img
end

histogram_equalization_ (generic function with 1 method)

In [17]:
# img = Float32.(dcm_parse(raw"F:\collected_dataset_for_ML\valid\image\SID-100809\1.2.840.113681.2206989178.1090.3561033058.289.dcm")[(0x7fe0, 0x0010)])
# mask = 1f0 .- zoom_pxiel_values(img)

# s = size(img)

# x = ceil(Int, s[1]/patch_size) + floor(Int, (s[1]-patch_size_half)/patch_size)
# y = ceil(Int, s[2]/patch_size) + floor(Int, (s[2]-patch_size_half)/patch_size)
# locations = []
# ct = 0
# for i = 1 : x-1
#     x_start = 1+(i-1)*patch_size_half
#     x_end = x_start+patch_size-1
#     for j = 1 : y-1
#         y_start = 1+(j-1)*patch_size_half
#         y_end = y_start+patch_size-1
#         # check patch
#         if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
#             # save patch
#             ct += 1
#             push!(locations, [x_start, x_end, y_start, y_end])
#             # img_patches[:, :, 1, ct] = (img[x_start:x_end, y_start:y_end])
#             # locations[ct] = (x_start, x_end, y_start, y_end)
#         end
#     end
#     # right col
#     y_start, y_end = s[2]-patch_size+1, s[2]
#     # check patch
#     if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
#         # save patch
#         ct += 1
#         push!(locations, [x_start, x_end, y_start, y_end])
#         # img_patches[:, :, 1, ct] = (img[x_start:x_end, y_start:y_end])
#         # locations[ct] = (x_start, x_end, y_start, y_end)
#     end
# end
# # last row
# x_start, x_end = s[1]-patch_size+1, s[1]
# for j = 1 : y-1
#     y_start = 1+(j-1)*patch_size_half
#     y_end = y_start+patch_size-1
#     # check patch
#     if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
#         # save patch
#         ct += 1
#         push!(locations, [x_start, x_end, y_start, y_end])
#         # img_patches[:, :, 1, ct] = (img[x_start:x_end, y_start:y_end])
#         # locations[ct] = (x_start, x_end, y_start, y_end)
#     end
# end
# # right col
# y_start, y_end = s[2]-patch_size+1, s[2]
# # check patch
# if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
#     # save patch
#     ct += 1
#     push!(locations, [x_start, x_end, y_start, y_end])
#     # img_patches[:, :, 1, ct] = (img[x_start:x_end, y_start:y_end])
#     # locations[ct] = (x_start, x_end, y_start, y_end)
# end
# s

In [18]:
# img_tmp = Gray.(1f0 .- zoom_pxiel_values(img))
# img_ = RGB.(img_tmp)
# for box in locations
#     x_start, x_end, y_start, y_end = box
#     color = RGB(rand(), rand(), rand())

#     img_[x_start, y_start:y_end] .= color  # Left vertical line
#     img_[x_end, y_start:y_end] .= color   # Right vertical line
#     img_[x_start:x_end, y_start] .= color  # Top horizontal line
#     img_[x_start:x_end, y_end] .= color    # Bottom horizontal line
# end
# save("image_patch_sample.png", img_)